In [1]:
import numpy as np
from keras import layers
from keras import models
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image_dataset_from_directory
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.regularizers import l2
from keras.utils import to_categorical
from keras.utils import multi_gpu_model
from tensorflow.distribute import MirroredStrategy
#from keras.models import load_model
import os
import matplotlib.pyplot as plt
import pickle


Path = "/home/ug-ml/felix-ML/AllDirections"

ChunkImage = "ChunkImage_"
ChunkLabel = "ChunkLabel_"
NameNumbers = [0, 1000]
MaxChunk = [101, 91]

rng = np.random.default_rng()

Chunk_Size = 8


Total = 0
for i in MaxChunk:
  Total+=i  

ShuffleIndex = np.zeros(Total, dtype = np.int)
IndexBatchAll = []
IndexBatch_i = []
for i in range(0, len(MaxChunk)):
    for j in range(0, MaxChunk[i]):
        IndexBatch_i.append(j + NameNumbers[i])
    IndexBatchAll.append(IndexBatch_i)
    IndexBatch_i = []

index = 0
for i in IndexBatchAll:
    for j in i:
        ShuffleIndex[index] = j
        index+=1

rng.shuffle(ShuffleIndex)
#print(ShuffleIndex)

RatioTrain = 0.9

TrainNo = int(Total * RatioTrain)
TrainIndex = ShuffleIndex[0:TrainNo]
ValIndex = ShuffleIndex[TrainNo:-1]

In [2]:

os.environ["TF_CPP_MIN_LOG_LEVEL"]="3"

strategy = MirroredStrategy()


PixelDimension = 36


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')


In [ ]:

with strategy.scope():

    model = models.Sequential()
    model.add(layers.SeparableConv2D(128, (3, 3), activation='relu', data_format='channels_first', input_shape=(PixelDimension, 128, 128)))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.SeparableConv2D(128, (3, 3), data_format='channels_first', activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.SeparableConv2D(128, (3, 3), data_format='channels_first',  activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Flatten())
    model.add(layers.Dropout(0.25))
    model.add(layers.Dense(512, activation='relu', kernel_regularizer = l2(0.0001)))
    model.add(layers.Dense(10, activation='softmax', kernel_regularizer = l2(0.0001)))



    model.compile(loss='categorical_crossentropy', optimizer=optimizers.RMSprop(learning_rate = 0.0005), metrics=['acc'])




#model.summary()


EarlyStop = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=20)
#Checkpoint = ModelCheckpoint("3DCovBest.hdf5", monitor='val_loss', verbose=1, save_best_only=True, mode='min', period=1)
Epochs = 5
TrainLoadedChunks = 10
EpochsChunk = int(Epochs * TrainIndex.shape[0] / TrainLoadedChunks)


BatchShuffle = np.arange(Chunk_Size * 10 * TrainLoadedChunks, dtype = int).reshape(TrainLoadedChunks * Chunk_Size * 10)
rng.shuffle(BatchShuffle)

index = 0
for i in range(0, EpochsChunk):

    train_images = np.zeros(Chunk_Size * 10 * 36 * TrainLoadedChunks * 128 * 128, dtype = np.float32).reshape(TrainLoadedChunks, Chunk_Size * 10, 36, 128, 128)
    train_lab = np.zeros(Chunk_Size * 10 * TrainLoadedChunks, dtype = np.int).reshape(TrainLoadedChunks, Chunk_Size * 10)
    for j in range(0, TrainLoadedChunks):
        index = i * TrainLoadedChunks + j
        TrainChunk_i = TrainIndex[index % TrainIndex.shape[0]]
        #print("Train Chunk", TrainChunk_i)




        train_images[j] = np.load(Path + "/" + ChunkImage + str(TrainChunk_i) + ".npy")
        train_lab[j] = np.load(Path + "/" + ChunkLabel + str(TrainChunk_i) + ".npy")

    train_images = train_images.reshape(-1, 36, 128, 128)
    train_images = train_images[BatchShuffle]
    train_lab = train_lab.reshape(-1)
    train_lab = to_categorical(train_lab[BatchShuffle])

    ValChunk_i = ValIndex[(i * TrainLoadedChunks) % ValIndex.shape[0]]
    #print("Val Chunk", ValChunk_i)

    val_images = np.load(Path + "/" + ChunkImage + str(ValChunk_i) + ".npy")
    val_lab = to_categorical(np.load(Path + "/" + ChunkLabel + str(ValChunk_i) + ".npy"))


    print("EPOCH: ", int(i / TrainIndex.shape[0]), "CHUNK: ", i % TrainIndex.shape[0])
    history = model.fit(train_images, train_lab, epochs=5, batch_size = 64, validation_data = (val_images, val_lab), shuffle = True, callbacks=[EarlyStop]) #, Checkpoint
    train_images = None
    train_lab = None
    val_images = None
    val_lab = None
    #model.save('1000AllDirection.h5')


EPOCH:  0 CHUNK:  0
Epoch 1/5
Instructions for updating:
Use `tf.data.Iterator.get_next_as_optional()` instead.
INFO:tensorflow:batch_all_reduce: 13 all-reduces with algorithm = nccl, num_packs = 1
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:batch_all_reduce: 13 all-reduces with algorithm = nccl, num_packs = 1
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:

Epoch 2/5
13/13 [==============================] - 4s 333ms/step - loss: 1.7682 - acc: 0.3862 - val_loss: 1.8377 - val_acc: 0.2750
Epoch 3/5
13/13 [==============================] - 4s 327ms/step - loss: 1.7370 - acc: 0.4225 - val_loss: 1.8645 - val_acc: 0.2750
Epoch 4/5
13/13 [==============================] - 4s 345ms/step - loss: 1.6850 - acc: 0.4112 - val_loss: 2.0972 - val_acc: 0.3000
Epoch 5/5
13/13 [==============================] - 4s 330ms/step - loss: 1.6464 - acc: 0.4487 - val_loss: 1.9330 - val_acc: 0.3375
EPOCH:  0 CHUNK:  10
Epoch 1/5
13/13 [==============================] - 5s 356ms/step - loss: 1.7076 - acc: 0.4062 - val_loss: 1.8572 - val_acc: 0.3875
Epoch 2/5
13/13 [==============================] - 4s 324ms/step - loss: 1.6412 - acc: 0.4375 - val_loss: 1.8613 - val_acc: 0.3000
Epoch 3/5
13/13 [==============================] - 4s 346ms/step - loss: 1.5772 - acc: 0.4650 - val_loss: 1.9212 - val_acc: 0.3750
Epoch 4/5
13/13 [==============================] - 4s 324ms/st

13/13 [==============================] - 4s 336ms/step - loss: 1.2887 - acc: 0.5950 - val_loss: 1.1964 - val_acc: 0.6375
Epoch 3/5
13/13 [==============================] - 4s 330ms/step - loss: 1.1048 - acc: 0.6787 - val_loss: 1.4900 - val_acc: 0.6000
Epoch 4/5
13/13 [==============================] - 4s 340ms/step - loss: 0.9872 - acc: 0.7050 - val_loss: 1.3194 - val_acc: 0.6125
Epoch 5/5
13/13 [==============================] - 4s 334ms/step - loss: 0.8357 - acc: 0.7713 - val_loss: 1.4086 - val_acc: 0.6375
EPOCH:  0 CHUNK:  22
Epoch 1/5
13/13 [==============================] - 5s 410ms/step - loss: 1.6127 - acc: 0.4938 - val_loss: 1.7621 - val_acc: 0.4000
Epoch 2/5
13/13 [==============================] - 4s 342ms/step - loss: 1.2513 - acc: 0.6150 - val_loss: 1.6293 - val_acc: 0.4500
Epoch 3/5
13/13 [==============================] - 4s 331ms/step - loss: 1.0750 - acc: 0.6850 - val_loss: 1.7494 - val_acc: 0.4125
Epoch 4/5
13/13 [==============================] - 4s 339ms/step - loss:

In [1]:
model.save('1000AllDirection.h5')




NameError: name 'model' is not defined

In [7]:
model.save('1000AllDirection.h5')